<a href="https://colab.research.google.com/github/ronwuyk/LSTM-learning/blob/master/rice%20is%20nice%20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
import sys
import pandas as pd
import keras
import seaborn as sns

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 9319562924630156142
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 14626652160
locality {
  bus_id: 1
  links {
  }
}
incarnation: 11688840955243427660
physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
xla_global_id: 416903419
]


In [3]:
print(f"TensorFlow Version: {tf.__version__}")
#print(f"Keras Version: {keras.__version__}")
print()
print(f"Python Version: {sys.version}")
print(f"Pandas Version: {pd.__version__}")
print(f"Numpy Version: {np.__version__}")
print(f"Seaborn Version: {sns.__version__}")
gpu = len(tf.config.list_physical_devices('GPU')) > 0
print()
print("GPU is", "PRESENT" if gpu else "NOT AVAILABLE")
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

TensorFlow Version: 2.15.0

Python Version: 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]
Pandas Version: 2.0.3
Numpy Version: 1.25.2
Seaborn Version: 0.13.1

GPU is PRESENT
Num GPUs Available:  1


In [4]:
!nvidia-smi

Tue Jul  2 02:05:51 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   66C    P0              31W /  70W |    103MiB / 15360MiB |      3%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [5]:
!uname -a

Linux 6686102731bf 6.1.85+ #1 SMP PREEMPT_DYNAMIC Tue Jun 18 14:18:04 UTC 2024 x86_64 x86_64 x86_64 GNU/Linux


## Adapting a CNN for Rice Variety Classification

This guide will walk you through adapting the Convolutional Neural Network (CNN) architecture from TensorFlow's image classification tutorial ([https://www.tensorflow.org/tutorials/images/classification](https://www.tensorflow.org/tutorials/images/classification)) to a new dataset containing images of five different rice varieties. You'll build and train the model to achieve at least 88% accuracy on the testing/validation set.

**Prerequisites:**

* Familiarity with TensorFlow and Keras for building deep learning models.
* Basic understanding of Convolutional Neural Networks (CNNs).
* Loaded rice variety image dataset with training, validation, and testing splits.

**Steps:**

1. **Import Libraries and Load Data:**

   - Start by importing the necessary libraries like TensorFlow, NumPy, and Matplotlib.
   - Load your pre-existing rice variety image dataset using TensorFlow's data loading functionalities (e.g., `tf.keras.preprocessing.image.ImageDataGenerator`). Remember to split the data into training, validation, and testing sets.

2. **Define the CNN Architecture:**

   - The provided TensorFlow tutorial uses a basic CNN architecture with convolutional and pooling layers followed by dense layers.
   - Modify this architecture to suit your specific dataset. Here are some considerations:
     - Experiment with the number and configuration of convolutional layers (filter size, number of filters, activation functions).
     - Consider adding techniques like batch normalization and dropout layers to improve model regularization.
     - Adjust the number of units in the dense layers based on the complexity of your data.

3. **Compile the Model:**

   - Once you've defined your CNN architecture, compile the model using a suitable optimizer (e.g., Adam), loss function (e.g., categorical cross-entropy for multi-class classification), and metrics (e.g., accuracy).

4. **Implement Training Loop with Early Stopping:**

   - Define a training loop that iterates through the training data in batches.
   - Inside the loop:
     - Feed the training data to the model for training.
     - Track training metrics (loss and accuracy) on each batch.
     - Evaluate the model's performance on the validation set periodically (e.g., after each epoch).
   - Implement an Early Stopping callback to halt training if the validation accuracy doesn't improve for a predefined number of epochs. This helps prevent overfitting.

5. **Train the Model:**

   - Train the model using the defined training loop. Monitor the training progress and validation accuracy.
   - If the validation accuracy plateaus or starts decreasing, adjust hyperparameters (learning rate, number of epochs) or modify the model architecture and retrain.

6. **Evaluate the Model:**

   - After training, evaluate the final model performance on the unseen testing set.
   - Ensure the testing accuracy meets or exceeds the target of 88%.

**Additional Tips:**

* Visualize the training and validation accuracy/loss curves to identify potential overfitting or underfitting issues.
* Consider data augmentation techniques to increase the size and diversity of your training data.
* Experiment with different learning rate schedules to optimize training speed and convergence.

By following these steps and iteratively refining your model architecture and training process, you should be able to achieve a CNN model with at least 88% accuracy for your rice variety classification task. Remember to adapt and modify these instructions based on your specific dataset and hardware limitations.


In [6]:
# data downloading
import pathlib

dataset_url = "/kaggle/input/Rice_Image_Dataset"
#data_dir = tf.keras.utils.get_file('rice_photos.tar', origin=dataset_url, extract=True)
data_dir = pathlib.Path(dataset_url).with_suffix('')

In [7]:
image_count = len(list(data_dir.glob('*/*.jpg')))
print(image_count)

0


In [8]:
batch_size = 32
img_height = 180
img_width = 180


In [10]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

NotFoundError: Could not find directory /kaggle/input/Rice_Image_Dataset

In [ ]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
class_names = train_ds.class_names
print(class_names)

In [ ]:
data_augmentation = keras.Sequential(
  [
    layers.RandomFlip("horizontal",
                      input_shape=(img_height,
                                  img_width,
                                  3)),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
  ]
)

In [ ]:
model = Sequential([
  data_augmentation,
  layers.Rescaling(1./255),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.2),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
#  layers.Dense(num_classes, name="outputs")
])

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
#training model
epochs = 10
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

In [ ]:
model.summary()

In [ ]:
# plot chart the accuracy and loss of the model
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
# predict new input

rice_url = "https://c8.alamy.com/comp/CXFFDA/organic-white-rice-sample-CXFFDA.jpg"
rice_path = tf.keras.utils.get_file('White_rice', rice_url)

img = tf.keras.utils.load_img(
    rice_path, target_size=(img_height, img_width)
)
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)